# LSTM

In [8]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torchmetrics
import matplotlib.pyplot as plt

In [9]:
DATA_PATH = r'C:\Users\connor\PycharmProjects\trading\data\analytics\analytics_voo.csv'
df = pd.read_csv(DATA_PATH, low_memory=False)

In [10]:
df.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'join_date', 'Id',
       'Date', 'DateLongDescription', 'DateShortDescription', 'DayLongName',
       'DayShortName', 'MonthLongName', 'MonthShortName', 'CalendarDay',
       'CalendarWeek', 'CalendarWeekStartDateId', 'CalendarWeekEndDateId',
       'CalendarDayInWeek', 'CalendarMonth', 'CalendarMonthStartDateId',
       'CalendarMonthEndDateId', 'CalendarNumberOfDaysInMonth',
       'CalendarDayInMonth', 'CalendarQuarter', 'CalendarQuarterStartDateId',
       'CalendarQuarterEndDateId', 'CalendarQuarterStartDate',
       'CalendarNumberOfDaysInQuarter', 'CalendarDayInQuarter', 'CalendarYear',
       'CalendarYearEndDateId', 'CalendarYearStartDate',
       'CalendarNumberOfDaysInYear', 'month_join_key', 'year_join_key',
       'seven_day_ema', 'CPALTT01USM657N', 'DFF', 'EXPINF10YR', 'GDPC1',
       'RSXFS', 'T10YFF', 'UNRATE', 'macd', 'macd_signal', 'macd_hist',
       'daily_obv', 'target'],
      dtype='object')

In [11]:
non_target_columns = ['open', 'high', 'low', 'close', 'volume', 'CPALTT01USM657N', 'DFF', 'EXPINF10YR', 'GDPC1', 'RSXFS', 'T10YFF', 'UNRATE', 'macd', 'macd_signal', 'macd_hist', 'daily_obv', 'seven_day_ema', 'close', 'target']

df_cols = df[non_target_columns]

# version with just close price and date
df_target_train = df[['date', 'target']].where(df.date <= '2020-01-01')
df_target_train.dropna(inplace=True)
df_target_test = df[['date', 'target']].where(df.date > '2020-01-01')
df_target_test.dropna(inplace=True)

# version with all columns
df_target_train_v2 = df[non_target_columns].where(df.date <= '2020-01-01')
df_target_train_v2.dropna(inplace=True)
df_target_test_v2 = df[non_target_columns].where(df.date > '2020-01-01')
df_target_test_v2.dropna(inplace=True)

In [12]:
print(df_target_train.columns)
print()
print(df_target_train_v2.columns)

Index(['date', 'target'], dtype='object')

Index(['open', 'high', 'low', 'close', 'volume', 'CPALTT01USM657N', 'DFF',
       'EXPINF10YR', 'GDPC1', 'RSXFS', 'T10YFF', 'UNRATE', 'macd',
       'macd_signal', 'macd_hist', 'daily_obv', 'seven_day_ema', 'close',
       'target'],
      dtype='object')


In [13]:
def create_sequences(df, seq_length, num_rows=len(df), include_all_features=False):
    """
    Input:
    df: pandas dataframe with at least 2 columns, a date column and a target column
    seq_length: How long the sequence should be. 
    num_rows: How many rows to use. Will pull all rows unless num_rows is provided, in which case the top num_rows will be pulled.
    include_all_features: Whether to use just the target column for the sequence, or to include all features.
        
    Output:
    Generates a sequence of seq_length length. It's shape is [num_rows, seq_length] if include_all_features is False, else [num_rows, seq_length * len(df.columns)]. 
    A sequence refers to how many should be appended as columns. 
    For example, if df = [['2020-01-01',100], ['2020-01-02', 101], ['2020-01-03', 102]] and seq_length = 2, the generated sequence will be [[101,102], [102,103]]. 
    """
    
    
    xs, ys = [], []
    
    if include_all_features == False:
        for itr in range(num_rows - seq_length):
            x = df.iloc[itr:(itr + seq_length), -1]
            y = df.iloc[itr + seq_length, -1]
            xs.append(x)
            ys.append(y)
    elif include_all_features == True:
        for itr in range(num_rows - seq_length):
            x = df.iloc[itr:(itr + seq_length), :]
            y = df.iloc[itr + seq_length, -1]
            xs.append(x)
            ys.append(y)

    else:
        print(f"error: include_all_features accepts True or False, got {include_all_features} instead.")
    return np.array(xs), np.array(ys)

In [14]:
# create training set
sequence_len = 10
num_rows = 50000
X_train, y_train = create_sequences(df_target_train, sequence_len, num_rows)
X_test, y_test = create_sequences(df_target_test, sequence_len, num_rows)

X_train_v2, y_train_v2 = create_sequences(df_target_train_v2, sequence_len, num_rows, include_all_features=True)
X_test_v2, y_test_v2 = create_sequences(df_target_test_v2, sequence_len, num_rows, include_all_features=True)

In [15]:
df[non_target_columns].head(n=100)

,open,high,low,close,volume,CPALTT01USM657N,DFF,EXPINF10YR,GDPC1,RSXFS,T10YFF,UNRATE,macd,macd_signal,macd_hist,daily_obv,seven_day_ema,close,target
0,83.935,84.042,83.820,83.967,232,0.12452,0.19,1.574237,16960.864,323990.0,2.48,9.4,1.1427,1.1947,-0.0521,-148100.0,81.6086,83.967,0
1,83.888,83.995,83.743,83.889,150,0.12452,0.19,1.574237,16960.864,323990.0,2.48,9.4,1.1427,1.1947,-0.0521,-148100.0,81.6086,83.889,1
2,83.888,83.995,83.774,83.920,1600,0.12452,0.19,1.574237,16960.864,323990.0,2.48,9.4,1.1427,1.1947,-0.0521,-148100.0,81.6086,83.920,1
3,83.904,84.011,83.776,83.923,100,0.12452,0.19,1.574237,16960.864,323990.0,2.48,9.4,1.1427,1.1947,-0.0521,-148100.0,81.6086,83.923,1
4,83.966,84.073,83.851,83.998,698,0.12452,0.19,1.574237,16960.864,323990.0,2.48,9.4,1.1427,1.1947,-0.0521,-148100.0,81.6086,83.998,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,83.577,83.684,83.463,83.609,50,0.12452,0.19,1.574237,16960.864,323990.0,2.56,9.4,1.1110,1.1834,-0.0724,-165200.0,81.7122,83.609,1
96,83.608,83.715,83.494,83.640,93,0.12452,0.19,1.574237,16960.864,323990.0,2.56,9.4,1.1110,1.1834,-0.0724,-165200.0,81.7122,83.640,0
97,83.593,83.700,83.479,83.625,100,0.12452,0.19,1.574237,16960.864,323990.0,2.56,9.4,1.1110,1.1834,-0.0724,-165200.0,81.7122,83.625,1
98,83.733,83.840,83.618,83.765,100,0.12452,0.19,1.574237,16960.864,323990.0,2.56,9.4,1.1110,1.1834,-0.0724,-165200.0,81.7122,83.765,1


In [16]:
X_train[0]

array([0., 1., 1., 1., 1., 1., 0., 1., 1., 0.])

In [17]:
y_train[0]

1.0

In [18]:
X_train_v2[0]

array([[ 8.39350000e+01,  8.40420000e+01,  8.38200000e+01,
         8.39670000e+01,  2.32000000e+02,  1.24519889e-01,
         1.90000000e-01,  1.57423670e+00,  1.69608640e+04,
         3.23990000e+05,  2.48000000e+00,  9.40000000e+00,
         1.14270000e+00,  1.19470000e+00, -5.21000000e-02,
        -1.48100000e+05,  8.16086000e+01,  8.39670000e+01,
         0.00000000e+00],
       [ 8.38880000e+01,  8.39950000e+01,  8.37430000e+01,
         8.38890000e+01,  1.50000000e+02,  1.24519889e-01,
         1.90000000e-01,  1.57423670e+00,  1.69608640e+04,
         3.23990000e+05,  2.48000000e+00,  9.40000000e+00,
         1.14270000e+00,  1.19470000e+00, -5.21000000e-02,
        -1.48100000e+05,  8.16086000e+01,  8.38890000e+01,
         1.00000000e+00],
       [ 8.38880000e+01,  8.39950000e+01,  8.37740000e+01,
         8.39200000e+01,  1.60000000e+03,  1.24519889e-01,
         1.90000000e-01,  1.57423670e+00,  1.69608640e+04,
         3.23990000e+05,  2.48000000e+00,  9.40000000e+00,
    

In [19]:
y_train_v2[0]

1.0

In [20]:
print(len(X_train), len(X_train_v2))
print(len(y_train), len(y_train_v2))

49990 49990
49990 49990


# using all fields available

In [21]:
# convert df into objects Torch can read
torch_X_train  = torch.from_numpy(X_train_v2).float()
torch_y_train = torch.from_numpy(y_train_v2).float()
torch_X_test = torch.from_numpy(X_test_v2).float()
torch_y_test = torch.from_numpy(y_test_v2).float()

# normalize data
torch_X_train = torch.nn.functional.normalize(torch_X_train)
torch_X_test = torch.nn.functional.normalize(torch_X_test)

print(torch_X_train.shape, torch_y_train.shape)
# create test and train sets
train_data_set = TensorDataset(torch_X_train, torch_y_train)
test_data_set = TensorDataset(torch_X_test, torch_y_test)

# confirm it works
sample = train_data_set[0]
input_sample, label_sample = sample
print('input sample:', input_sample)
print('label sample:', label_sample)

torch.Size([49990, 10, 19]) torch.Size([49990])
input sample: tensor([[ 0.3158,  0.3158,  0.3158,  0.3158,  0.1254,  0.3162,  0.3162,  0.3162,
          0.3162,  0.3162,  0.3162,  0.3162,  0.3162,  0.3162, -0.3162, -0.3162,
          0.3162,  0.3158,  0.0000],
        [ 0.3156,  0.3156,  0.3155,  0.3155,  0.0811,  0.3162,  0.3162,  0.3162,
          0.3162,  0.3162,  0.3162,  0.3162,  0.3162,  0.3162, -0.3162, -0.3162,
          0.3162,  0.3155,  0.3780],
        [ 0.3156,  0.3156,  0.3156,  0.3156,  0.8650,  0.3162,  0.3162,  0.3162,
          0.3162,  0.3162,  0.3162,  0.3162,  0.3162,  0.3162, -0.3162, -0.3162,
          0.3162,  0.3156,  0.3780],
        [ 0.3157,  0.3157,  0.3156,  0.3156,  0.0541,  0.3162,  0.3162,  0.3162,
          0.3162,  0.3162,  0.3162,  0.3162,  0.3162,  0.3162, -0.3162, -0.3162,
          0.3162,  0.3156,  0.3780],
        [ 0.3159,  0.3159,  0.3159,  0.3159,  0.3774,  0.3162,  0.3162,  0.3162,
          0.3162,  0.3162,  0.3162,  0.3162,  0.3162,  0.3162

# Using just the close price

In [22]:
# # convert df into objects Torch can read
# torch_X_train  = torch.from_numpy(X_train).float()
# torch_y_train = torch.from_numpy(y_train).float()
# torch_X_test = torch.from_numpy(X_test).float()
# torch_y_test = torch.from_numpy(y_test).float()
# 
# # create test and train sets
# train_data_set = TensorDataset(torch_X_train, torch_y_train)
# test_data_set = TensorDataset(torch_X_test, torch_y_test)
# 
# # confirm it works
# sample = train_data_set[0]
# input_sample, label_sample = sample
# print('input sample:', input_sample)
# print('label sample:', label_sample)

In [23]:
print(input_sample.shape)

torch.Size([10, 19])


In [24]:
num_features = len(df_target_train_v2.columns)
batch_size = 1000
shuffle = False
hidden_size = round(num_features / 2)
num_layers = 2
dropout = .5

learning_rate = 0.0001
num_epochs = 50

# create dataloader
train_dataloader = DataLoader(train_data_set, batch_size=batch_size, shuffle=shuffle, drop_last=True)
test_dataloader = DataLoader(test_data_set, batch_size=batch_size, shuffle=shuffle, drop_last=True)
# test loader
# x, y = next(iter(train_dataloader))
# 
# print('x', x, 'y', y)

In [25]:
# LTSM
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__() #super makes all the methods available in nn.Module available for the new class Net
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )
        self.fc = nn.Linear(in_features=hidden_size, out_features=1)
        self.sigmoid = nn.Sigmoid() # we want a binary output, not %

    def forward(self, x):
        h0 = torch.zeros(num_layers, x.size(0), hidden_size)
        c0 = torch.zeros(num_layers, x.size(0), hidden_size)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out

In [27]:
net = Net(input_size=num_features)
criterion = nn.BCELoss(reduction='sum') # for binary prediction. Using the 'target' column
#criterion = nn.MSELoss() # for regression, predicting the 'close' column
optimizer = optim.Adam(
    net.parameters(), lr=learning_rate
)
acc = torchmetrics.Accuracy(task="binary")

for epoch in range(num_epochs):
    for dummy, data in enumerate(train_dataloader): # enumerate ensures that the dataloader always starts at the beginning since it's a generator object
        seqs = data[0].view(batch_size, sequence_len, num_features)
        outputs = net(seqs).squeeze()
        loss = criterion(outputs, data[1])
        acc(torch.round(outputs), data[1])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}, Accuracy: {acc.compute()}")
    # early break if loss isn't changing beyond the learning rate
    if loss.item() < .1:
        break

Epoch 1, Loss: 691.9000854492188, Accuracy: 0.5504081845283508
Epoch 2, Loss: 691.5743408203125, Accuracy: 0.5517041087150574
Epoch 3, Loss: 690.932373046875, Accuracy: 0.5521292686462402
Epoch 4, Loss: 690.681396484375, Accuracy: 0.5523418188095093
Epoch 5, Loss: 690.2191162109375, Accuracy: 0.5524693727493286
Epoch 6, Loss: 691.055908203125, Accuracy: 0.5525544285774231
Epoch 7, Loss: 691.0161743164062, Accuracy: 0.5526151657104492
Epoch 8, Loss: 690.7001342773438, Accuracy: 0.5526607036590576
Epoch 9, Loss: 690.25390625, Accuracy: 0.552696168422699
Epoch 10, Loss: 690.836669921875, Accuracy: 0.5527244806289673
Epoch 11, Loss: 690.8641967773438, Accuracy: 0.5527476668357849
Epoch 12, Loss: 690.9657592773438, Accuracy: 0.5527669787406921
Epoch 13, Loss: 691.8514404296875, Accuracy: 0.5527833700180054
Epoch 14, Loss: 691.0157470703125, Accuracy: 0.5527973771095276
Epoch 15, Loss: 690.7243041992188, Accuracy: 0.5528095364570618
Epoch 16, Loss: 690.8662719726562, Accuracy: 0.552820146083

In [ ]:
# param_grid = {
#     'hidden_size': [5, 10, num_features]
#     , 'num_layers': range(4)
#     , 'dropout': [.1, .5, .9]
#     , 'learning_rate': [.0001, .0005, .001]
#     , 'num_epochs': [10, 25, 50]}
#
# best_params = {'hidden_size': 0, 'num_layers': 0, 'dropout': 0, 'learning_rate': 0, 'num_epochs': 0}

In [ ]:
# # Define MSE metric
# mse = torchmetrics.regression.MeanSquaredError()
# 
# net.eval()
# with torch.no_grad():
#     for seqs, labels in test_dataloader:
#         seqs = seqs.view(batch_size, torch_X_test.shape[1], num_features)
#         # Pass seqs to net and squeeze the result
#         outputs = net(seqs).squeeze()
#         mse(outputs, labels)
# 
# # Compute final metric value
# test_mse = mse.compute()
# print(f"Test MSE: {test_mse}")
# print(f"Test RMSE: {test_mse**.5}")

In [28]:
# test score

net.eval()
with torch.no_grad():
    for dummy, data in enumerate(test_dataloader):
        seqs = data[0].view(batch_size, sequence_len, num_features)
        outputs = net(data[0]).squeeze()
        acc(torch.round(outputs), data[1])

print(f"Test accuracy score: {acc.compute()}")

Test accuracy score: 0.5520796179771423


# Results

Baseline guessing 

In [29]:
random_guess_likelihood = df.target.mean()
# mean = percent the candle was higher than the previous candle

print('Random guess likelihood', random_guess_likelihood)

Random guess likelihood 0.4687579112426077


# Writeup
At the minute level, 46.875% of the time the next close price is higher than the previous close price for VOO.
The model is able to correctly predict a candle 56.3131% of the time making it better than a random (weighted) coinflip. The thing holding the model back is that it doesn't take into account expected payoff. The average positive candel is higher than the average negative candle in absolute terms. The model is doing almost exactly as well on the train and test set so there is not any overfitting going on. The accuracy is a bit low though so even though it's a nn it has high bias and underfitting overall.


I used the following hyperparameters:
sequence_len = 10
* Sequence of length 10 seemed to give sufficient information without overwhelming my computer. Each input becomes sequence_len * feature so the compute cost is quite high increasing this value
num_rows = 100000
* This is to limit how much data I'm using. Adding more data doesn't seem to improve accuracy but it does crash performance.
batch_size = 1000
* Mostly arbitarily chosen. 1000 makes the model run faster and doesn't seem to sacrifice accuracy either.
shuffle = False
* don't want to shuffle sequence data since that defeats the purpose
hidden_size = round(num_features / 2)
* Heuristic that the best hidden_size is based on the average between the input and output sizes. Here my output is just 1 so it's just the number of features divided by 2
num_layers = 2
* Again heuristic, 2 hidden layers should be able to solve anything
dropout = .5
* Another mostly random chosen value. Playing around with this doesn't impact the model too much.

learning_rate = 0.0001
* small enough to change directions but not too small to where it doesn't update
num_epochs = 50
* Probably don't need this many. It stops really changing much after like 3-4 epochs


I chose not to do dimensionality reduction for the LSTM model. I'm allowing the model to figure out which features and relationships are important or not. A big part of the value of deep learning is that it doesn't require me doing any sort of feature engineering or reduction.


# LSTM Tuning


In [30]:
def train_and_evaluate(model, num_epochs, batch_size):
    criterion = nn.BCELoss(reduction='sum') # for binary prediction. Using the 'target' column
    #criterion = nn.MSELoss() # for regression, predicting the 'close' column
    optimizer = optim.Adam(net.parameters(), lr=lr)
    for epoch in range(num_epochs):
        for dummy, data in enumerate(train_dataloader):
            seqs = data[0].view(batch_size, sequence_len, num_features)
            outputs = model(data[0]).squeeze()
            loss = criterion(outputs, data[1])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
    # don't update the weights after each batch        
    model.eval()
    with torch.no_grad():
        for dummy, data in enumerate(test_dataloader):
            seqs = data[0].view(batch_size, sequence_len, num_features)
            outputs = model(data[0]).squeeze()
            acc(torch.round(outputs), data[1])
    
    return acc.compute().numpy()

In [31]:
learning_rates = [.00001, .00005, .0001]
batch_sizes = [1000, 5000]
epochs = [2, 3, 5, 10, 25]

In [ ]:
best_score = 0
best_params = None
for lr in learning_rates:
    for bs in batch_sizes:
        for ne in epochs:
            # Create DataLoaders with current batch_size
            train_dataloader = DataLoader(train_data_set, batch_size=bs, shuffle=shuffle, drop_last=True)
            test_dataloader = DataLoader(test_data_set, batch_size=bs, shuffle=shuffle, drop_last=True)
            
            model = Net(input_size=num_features)
            current_score = train_and_evaluate(model, ne, bs)

            # Update best_score and best_params if current_score is better
            print('learning_rate:', lr, 'batch_size:', bs, 'ne:', ne, 'current_score:', current_score)
            if current_score > best_score:
                best_score = current_score
                best_params = {'learning_rate': lr, 'batch_size': bs, 'num_epochs': ne}

print("Best hyperparameters:", best_params)
print("Best score:", best_score) 

learning_rate: 1e-05 batch_size: 1000 ne: 2 current_score: 0.55126333
learning_rate: 1e-05 batch_size: 1000 ne: 3 current_score: 0.55011433
learning_rate: 1e-05 batch_size: 1000 ne: 5 current_score: 0.54900795
learning_rate: 1e-05 batch_size: 1000 ne: 10 current_score: 0.54794174
learning_rate: 1e-05 batch_size: 1000 ne: 25 current_score: 0.5469136
learning_rate: 1e-05 batch_size: 5000 ne: 2 current_score: 0.54631984
learning_rate: 1e-05 batch_size: 5000 ne: 3 current_score: 0.5454238
learning_rate: 1e-05 batch_size: 5000 ne: 5 current_score: 0.5448718
learning_rate: 1e-05 batch_size: 5000 ne: 10 current_score: 0.54433686


In [ ]:
def plot_best_model(model, best_params, test_dataloader):
    net = Net(input_size=num_features)
    criterion = nn.BCELoss(reduction='sum') # for binary prediction. Using the 'target' column
    #criterion = nn.MSELoss() # for regression, predicting the 'close' column
    optimizer = optim.Adam(
        net.parameters(), lr=best_params['learning_rate']
    )
    
    predictions = []
    
    for epoch in range(best_params['num_epochs']):
        for seqs, labels in enumerate(train_dataloader):
            seqs = seqs.view(best_params['batch_size'], sequence_len, num_features)
            outputs = net(seqs).squeeze()
            predictions.append(outputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
    test_data_df = pd.DataFrame(list(test_dataloader))
    test_data_df.plot()
    predictions_df = pd.DataFrame(predictions)
    predictions_df.plot()
    plt.show()
    

# Final Report

Parameter tuning does very little to improve the model. The only thing that seems to make a difference is using a smaller number of epochs. To make this project shine a bit more, future work should include additional data gathering, especially data that isn't already strongly correlated with the existing features. There is also room to change this from a binary classification task to a regression task. What is currently lost is that not all positive or negative candles have the same magnitutde. Simply predicting up or down might have worse value than either a regression task or a multi classifier that could improve the rate of return on investments. Additionally, having a more zoomed out look might actually be more valuable. Instead of predicting each minute candle, predicting each hour or day may yield better results. I should also include training data much closer to the current year. I split based on pre 2020 and post 2020 which makes modern predictions less useful. 